# Dependency Injection Examples for Deadline 2

This notebook demonstrates **meaningful** vs **trivial** usage of Dependency Injection patterns in C# applications.

## 🎯 **Learning Objectives**
- Understand when Dependency Injection provides real benefits
- Learn proper DI container configuration and usage
- Avoid DI anti-patterns and overuse
- Implement DI that improves testability and maintainability

## ✅ **Meaningful Dependency Injection Usage Examples**

### 1. Service Layer with Proper DI Configuration

In [ ]:
// ✅ MEANINGFUL: Proper service abstractions with DI
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Configuration;

// Domain interfaces
public interface IUserRepository
{
    Task<User> GetByIdAsync(int id);
    Task<IEnumerable<User>> GetActiveUsersAsync();
    Task SaveAsync(User user);
}

public interface IEmailService
{
    Task SendWelcomeEmailAsync(User user);
    Task SendPasswordResetAsync(User user, string resetToken);
}

public interface IUserService
{
    Task<User> CreateUserAsync(string email, string password);
    Task<bool> ResetPasswordAsync(string email);
    Task<UserStatistics> GetUserStatisticsAsync();
}

// Concrete implementations
public class SqlUserRepository : IUserRepository
{
    private readonly string _connectionString;
    private readonly ILogger<SqlUserRepository> _logger;
    
    public SqlUserRepository(IConfiguration configuration, ILogger<SqlUserRepository> logger)
    {
        _connectionString = configuration.GetConnectionString("DefaultConnection");
        _logger = logger;
    }
    
    public async Task<User> GetByIdAsync(int id)
    {
        _logger.LogInformation("Fetching user with ID: {UserId}", id);
        // Database implementation
        await Task.Delay(100); // Simulate DB call
        return new User { Id = id, Email = $"user{id}@example.com", IsActive = true };
    }
    
    public async Task<IEnumerable<User>> GetActiveUsersAsync()
    {
        _logger.LogInformation("Fetching all active users");
        await Task.Delay(200); // Simulate DB call
        return new List<User>
        {
            new User { Id = 1, Email = "user1@example.com", IsActive = true },
            new User { Id = 2, Email = "user2@example.com", IsActive = true }
        };
    }
    
    public async Task SaveAsync(User user)
    {
        _logger.LogInformation("Saving user: {UserEmail}", user.Email);
        await Task.Delay(150); // Simulate DB call
    }
}

public class SmtpEmailService : IEmailService
{
    private readonly EmailConfiguration _config;
    private readonly ILogger<SmtpEmailService> _logger;
    
    public SmtpEmailService(EmailConfiguration config, ILogger<SmtpEmailService> logger)
    {
        _config = config;
        _logger = logger;
    }
    
    public async Task SendWelcomeEmailAsync(User user)
    {
        _logger.LogInformation("Sending welcome email to: {UserEmail}", user.Email);
        // SMTP implementation
        await Task.Delay(300); // Simulate email sending
        Console.WriteLine($"Welcome email sent to {user.Email}");
    }
    
    public async Task SendPasswordResetAsync(User user, string resetToken)
    {
        _logger.LogInformation("Sending password reset email to: {UserEmail}", user.Email);
        await Task.Delay(300); // Simulate email sending
        Console.WriteLine($"Password reset email sent to {user.Email} with token: {resetToken}");
    }
}

// Business service with multiple dependencies
public class UserService : IUserService
{
    private readonly IUserRepository _userRepository;
    private readonly IEmailService _emailService;
    private readonly ILogger<UserService> _logger;
    
    public UserService(
        IUserRepository userRepository,
        IEmailService emailService,
        ILogger<UserService> logger)
    {
        _userRepository = userRepository ?? throw new ArgumentNullException(nameof(userRepository));
        _emailService = emailService ?? throw new ArgumentNullException(nameof(emailService));
        _logger = logger ?? throw new ArgumentNullException(nameof(logger));
    }
    
    public async Task<User> CreateUserAsync(string email, string password)
    {
        _logger.LogInformation("Creating new user: {Email}", email);
        
        try
        {
            var user = new User
            {
                Email = email,
                PasswordHash = HashPassword(password),
                CreatedAt = DateTime.UtcNow,
                IsActive = true
            };
            
            await _userRepository.SaveAsync(user);
            await _emailService.SendWelcomeEmailAsync(user);
            
            _logger.LogInformation("User created successfully: {Email}", email);
            return user;
        }
        catch (Exception ex)
        {
            _logger.LogError(ex, "Failed to create user: {Email}", email);
            throw;
        }
    }
    
    public async Task<bool> ResetPasswordAsync(string email)
    {
        _logger.LogInformation("Initiating password reset for: {Email}", email);
        
        try
        {
            // In real implementation, you'd search by email
            var user = await _userRepository.GetByIdAsync(1); // Simplified
            
            if (user == null || !user.IsActive)
            {
                _logger.LogWarning("Password reset attempted for non-existent or inactive user: {Email}", email);
                return false;
            }
            
            var resetToken = GenerateResetToken();
            await _emailService.SendPasswordResetAsync(user, resetToken);
            
            _logger.LogInformation("Password reset email sent for: {Email}", email);
            return true;
        }
        catch (Exception ex)
        {
            _logger.LogError(ex, "Failed to process password reset for: {Email}", email);
            return false;
        }
    }
    
    public async Task<UserStatistics> GetUserStatisticsAsync()
    {
        _logger.LogInformation("Calculating user statistics");
        
        var users = await _userRepository.GetActiveUsersAsync();
        
        return new UserStatistics
        {
            TotalActiveUsers = users.Count(),
            NewUsersThisMonth = users.Count(u => u.CreatedAt > DateTime.UtcNow.AddDays(-30)),
            CalculatedAt = DateTime.UtcNow
        };
    }
    
    private string HashPassword(string password) => $"hashed_{password}"; // Simplified
    private string GenerateResetToken() => Guid.NewGuid().ToString("N")[..8];
}

// Supporting classes
public class User
{
    public int Id { get; set; }
    public string Email { get; set; }
    public string PasswordHash { get; set; }
    public DateTime CreatedAt { get; set; }
    public bool IsActive { get; set; }
}

public class UserStatistics
{
    public int TotalActiveUsers { get; set; }
    public int NewUsersThisMonth { get; set; }
    public DateTime CalculatedAt { get; set; }
}

public class EmailConfiguration
{
    public string SmtpServer { get; set; }
    public int Port { get; set; }
    public string Username { get; set; }
    public string Password { get; set; }
}

### 2. DI Container Configuration (Program.cs / Startup.cs)

In [ ]:
// ✅ MEANINGFUL: Proper DI container setup with different lifetimes
public class Startup
{
    private readonly IConfiguration _configuration;
    
    public Startup(IConfiguration configuration)
    {
        _configuration = configuration;
    }
    
    public void ConfigureServices(IServiceCollection services)
    {
        // Configuration objects - Singleton lifetime
        services.Configure<EmailConfiguration>(_configuration.GetSection("Email"));
        services.AddSingleton<EmailConfiguration>(provider =>
            provider.GetRequiredService<IOptions<EmailConfiguration>>().Value);
        
        // Infrastructure services - Singleton lifetime (stateless, expensive to create)
        services.AddSingleton<ILogger>(provider => 
            provider.GetRequiredService<ILoggerFactory>().CreateLogger("Application"));
        
        // Data access - Scoped lifetime (per request in web apps)
        services.AddScoped<IUserRepository, SqlUserRepository>();
        
        // External services - Scoped lifetime (may hold state per operation)
        services.AddScoped<IEmailService, SmtpEmailService>();
        
        // Business services - Scoped lifetime (typical for business logic)
        services.AddScoped<IUserService, UserService>();
        
        // Add specific implementations for different environments
        if (_configuration.GetValue<bool>("UseTestingServices"))
        {
            services.AddScoped<IEmailService, TestEmailService>();
        }
        
        // Factory pattern for complex object creation
        services.AddSingleton<IServiceFactory, ServiceFactory>();
        
        // Decorator pattern example
        services.Decorate<IUserService, CachedUserService>();
        services.Decorate<IUserService, TimedUserService>();
        
        // Add HttpClient for external API calls
        services.AddHttpClient<ExternalApiService>(client =>
        {
            client.BaseAddress = new Uri(_configuration["ExternalApi:BaseUrl"]);
            client.Timeout = TimeSpan.FromSeconds(30);
        });
    }
}

// Test implementation for email service
public class TestEmailService : IEmailService
{
    private readonly ILogger<TestEmailService> _logger;
    
    public TestEmailService(ILogger<TestEmailService> logger)
    {
        _logger = logger;
    }
    
    public async Task SendWelcomeEmailAsync(User user)
    {
        _logger.LogInformation("TEST: Would send welcome email to {Email}", user.Email);
        await Task.CompletedTask;
    }
    
    public async Task SendPasswordResetAsync(User user, string resetToken)
    {
        _logger.LogInformation("TEST: Would send reset email to {Email} with token {Token}", 
            user.Email, resetToken);
        await Task.CompletedTask;
    }
}


### 3. Use Factory pattern for complex service creation

In [ ]:

public interface IServiceFactory
{
    IPaymentProcessor CreatePaymentProcessor(PaymentMethod method);
    INotificationService CreateNotificationService(NotificationType type);
}

public class ServiceFactory : IServiceFactory
{
    private readonly IServiceProvider _serviceProvider;
    
    public ServiceFactory(IServiceProvider serviceProvider)
    {
        _serviceProvider = serviceProvider;
    }
    
    public IPaymentProcessor CreatePaymentProcessor(PaymentMethod method)
    {
        return method switch
        {
            PaymentMethod.CreditCard => _serviceProvider.GetRequiredService<CreditCardProcessor>(),
            PaymentMethod.PayPal => _serviceProvider.GetRequiredService<PayPalProcessor>(),
            PaymentMethod.BankTransfer => _serviceProvider.GetRequiredService<BankTransferProcessor>(),
            _ => throw new NotSupportedException($"Payment method {method} not supported")
        };
    }
    
    public INotificationService CreateNotificationService(NotificationType type)
    {
        return type switch
        {
            NotificationType.Email => _serviceProvider.GetRequiredService<EmailNotificationService>(),
            NotificationType.SMS => _serviceProvider.GetRequiredService<SmsNotificationService>(),
            NotificationType.Push => _serviceProvider.GetRequiredService<PushNotificationService>(),
            _ => throw new NotSupportedException($"Notification type {type} not supported")
        };
    }
}

// Supporting enums and interfaces
public enum PaymentMethod { CreditCard, PayPal, BankTransfer }
public enum NotificationType { Email, SMS, Push }
public interface IPaymentProcessor { }
public interface INotificationService { }
public class CreditCardProcessor : IPaymentProcessor { }
public class PayPalProcessor : IPaymentProcessor { }
public class BankTransferProcessor : IPaymentProcessor { }
public class EmailNotificationService : INotificationService { }
public class SmsNotificationService : INotificationService { }
public class PushNotificationService : INotificationService { }
public class ExternalApiService { }

### 4. Decorator Pattern with DI for Cross-Cutting Concerns

In [ ]:
// ✅ MEANINGFUL: Using decorators for cross-cutting concerns
using System.Diagnostics;
using Microsoft.Extensions.Caching.Memory;

// Caching decorator
public class CachedUserService : IUserService
{
    private readonly IUserService _innerService;
    private readonly IMemoryCache _cache;
    private readonly ILogger<CachedUserService> _logger;
    private readonly TimeSpan _cacheExpiration = TimeSpan.FromMinutes(15);
    
    public CachedUserService(
        IUserService innerService,
        IMemoryCache cache,
        ILogger<CachedUserService> logger)
    {
        _innerService = innerService;
        _cache = cache;
        _logger = logger;
    }
    
    public async Task<User> CreateUserAsync(string email, string password)
    {
        // Don't cache write operations
        var result = await _innerService.CreateUserAsync(email, password);
        
        // Invalidate related cache entries
        _cache.Remove("user_statistics");
        
        return result;
    }
    
    public async Task<bool> ResetPasswordAsync(string email)
    {
        // Don't cache write operations
        return await _innerService.ResetPasswordAsync(email);
    }
    
    public async Task<UserStatistics> GetUserStatisticsAsync()
    {
        const string cacheKey = "user_statistics";
        
        if (_cache.TryGetValue(cacheKey, out UserStatistics cachedStats))
        {
            _logger.LogInformation("User statistics retrieved from cache");
            return cachedStats;
        }
        
        _logger.LogInformation("User statistics not in cache, fetching from service");
        var stats = await _innerService.GetUserStatisticsAsync();
        
        _cache.Set(cacheKey, stats, _cacheExpiration);
        _logger.LogInformation("User statistics cached for {Expiration}", _cacheExpiration);
        
        return stats;
    }
}

// Performance timing decorator
public class TimedUserService : IUserService
{
    private readonly IUserService _innerService;
    private readonly ILogger<TimedUserService> _logger;
    
    public TimedUserService(IUserService innerService, ILogger<TimedUserService> logger)
    {
        _innerService = innerService;
        _logger = logger;
    }
    
    public async Task<User> CreateUserAsync(string email, string password)
    {
        return await ExecuteWithTiming(
            () => _innerService.CreateUserAsync(email, password),
            "CreateUser",
            new { email });
    }
    
    public async Task<bool> ResetPasswordAsync(string email)
    {
        return await ExecuteWithTiming(
            () => _innerService.ResetPasswordAsync(email),
            "ResetPassword",
            new { email });
    }
    
    public async Task<UserStatistics> GetUserStatisticsAsync()
    {
        return await ExecuteWithTiming(
            () => _innerService.GetUserStatisticsAsync(),
            "GetUserStatistics",
            null);
    }
    
    private async Task<T> ExecuteWithTiming<T>(
        Func<Task<T>> operation,
        string operationName,
        object parameters)
    {
        var stopwatch = Stopwatch.StartNew();
        
        try
        {
            _logger.LogInformation("Starting {Operation} with parameters: {@Parameters}", 
                operationName, parameters);
            
            var result = await operation();
            
            stopwatch.Stop();
            _logger.LogInformation("Completed {Operation} in {ElapsedMs}ms", 
                operationName, stopwatch.ElapsedMilliseconds);
            
            return result;
        }
        catch (Exception ex)
        {
            stopwatch.Stop();
            _logger.LogError(ex, "Failed {Operation} after {ElapsedMs}ms", 
                operationName, stopwatch.ElapsedMilliseconds);
            throw;
        }
    }
}

// Extension method for decorator registration
public static class ServiceCollectionExtensions
{
    public static IServiceCollection Decorate<TInterface, TDecorator>(this IServiceCollection services)
        where TDecorator : class, TInterface
    {
        // Get the existing service descriptor
        var descriptor = services.FirstOrDefault(s => s.ServiceType == typeof(TInterface));
        if (descriptor == null)
        {
            throw new InvalidOperationException($"Service {typeof(TInterface).Name} is not registered");
        }
        
        // Remove the original registration
        services.Remove(descriptor);
        
        // Register the original implementation with a different key
        services.Add(new ServiceDescriptor(
            typeof(TInterface),
            provider => CreateInnerService(provider, descriptor),
            descriptor.Lifetime));
        
        // Register the decorator
        services.Add(new ServiceDescriptor(
            typeof(TInterface),
            typeof(TDecorator),
            descriptor.Lifetime));
        
        return services;
    }
    
    private static object CreateInnerService(IServiceProvider provider, ServiceDescriptor descriptor)
    {
        if (descriptor.ImplementationInstance != null)
            return descriptor.ImplementationInstance;
        
        if (descriptor.ImplementationFactory != null)
            return descriptor.ImplementationFactory(provider);
        
        return ActivatorUtilities.CreateInstance(provider, descriptor.ImplementationType);
    }
}

## ❌ **Trivial/Incorrect Dependency Injection Usage Examples**

### Anti-patterns to avoid:

In [ ]:
// ❌ TRIVIAL: Service Locator anti-pattern
public class BadServiceLocatorUsage
{
    private readonly IServiceProvider _serviceProvider;
    
    public BadServiceLocatorUsage(IServiceProvider serviceProvider)
    {
        _serviceProvider = serviceProvider; // Don't inject IServiceProvider!
    }
    
    public void ProcessData()
    {
        // This hides dependencies and makes testing difficult
        var userService = _serviceProvider.GetService<IUserService>();
        var emailService = _serviceProvider.GetService<IEmailService>();
        
        // Business logic...
    }
}

// ❌ UNNECESSARY: Using DI for simple value types
public class BadValueTypeInjection
{
    private readonly int _maxRetries;
    private readonly string _apiKey;
    
    // Don't inject simple values that don't change
    public BadValueTypeInjection(int maxRetries, string apiKey)
    {
        _maxRetries = maxRetries;
        _apiKey = apiKey;
    }
}

// ✅ BETTER: Use configuration objects
public class BetterConfigurationUsage
{
    private readonly ApiConfiguration _config;
    
    public BetterConfigurationUsage(ApiConfiguration config)
    {
        _config = config;
    }
}

public class ApiConfiguration
{
    public int MaxRetries { get; set; }
    public string ApiKey { get; set; }
    public TimeSpan Timeout { get; set; }
}

// ❌ OVERUSE: Injecting everything unnecessarily
public class OveruseDIExample
{
    // Too many dependencies - this class probably has too many responsibilities
    public OveruseDIExample(
        IService1 service1,
        IService2 service2,
        IService3 service3,
        IService4 service4,
        IService5 service5,
        IService6 service6,
        IService7 service7,
        ILogger logger,
        IConfiguration config)
    {
        // This suggests the class violates Single Responsibility Principle
    }
}

// ❌ WRONG LIFETIME: Using wrong service lifetimes
public void BadLifetimeConfiguration(IServiceCollection services)
{
    // ❌ DbContext as Singleton - causes threading issues
    services.AddSingleton<DbContext, ApplicationDbContext>();
    
    // ❌ Expensive service as Transient - creates too many instances
    services.AddTransient<ExpensiveExternalApiClient>();
    
    // ❌ Stateful service as Singleton - causes state sharing issues
    services.AddSingleton<IUserSessionManager, UserSessionManager>();
}

// ❌ NEW KEYWORD OVERUSE: Not using DI where it would help
public class BadNewKeywordUsage
{
    private readonly ILogger _logger;
    
    public BadNewKeywordUsage(ILogger logger)
    {
        _logger = logger;
    }
    
    public async Task ProcessAsync()
    {
        // ❌ Creating dependencies manually instead of injecting
        var httpClient = new HttpClient(); // Should be injected
        var userRepository = new SqlUserRepository(/* ... */); // Should be injected
        
        // This makes testing difficult and violates DI principles
    }
}

// Supporting interfaces for examples
public interface IService1 { }
public interface IService2 { }
public interface IService3 { }
public interface IService4 { }
public interface IService5 { }
public interface IService6 { }
public interface IService7 { }
public class ExpensiveExternalApiClient { }
public interface IUserSessionManager { }
public class UserSessionManager : IUserSessionManager { }

## 🔧 **Best Practices for Dependency Injection**

### 1. Constructor Injection Patterns

In [ ]:
// ✅ GOOD: Proper constructor injection with validation
public class ProperConstructorInjection
{
    private readonly IUserRepository _userRepository;
    private readonly IEmailService _emailService;
    private readonly ILogger<ProperConstructorInjection> _logger;
    
    public ProperConstructorInjection(
        IUserRepository userRepository,
        IEmailService emailService,
        ILogger<ProperConstructorInjection> logger)
    {
        // Always validate constructor parameters
        _userRepository = userRepository ?? throw new ArgumentNullException(nameof(userRepository));
        _emailService = emailService ?? throw new ArgumentNullException(nameof(emailService));
        _logger = logger ?? throw new ArgumentNullException(nameof(logger));
    }
    
    // ✅ Alternative validation using Guard clauses
    public ProperConstructorInjection(
        IUserRepository userRepository,
        IEmailService emailService,
        ILogger<ProperConstructorInjection> logger,
        bool useGuardClauses)
    {
        Guard.Against.Null(userRepository, nameof(userRepository));
        Guard.Against.Null(emailService, nameof(emailService));
        Guard.Against.Null(logger, nameof(logger));
        
        _userRepository = userRepository;
        _emailService = emailService;
        _logger = logger;
    }
}

// ✅ GOOD: Using primary constructors (C# 12+)
public class ModernConstructorInjection(
    IUserRepository userRepository,
    IEmailService emailService,
    ILogger<ModernConstructorInjection> logger)
{
    // Primary constructor automatically creates readonly fields
    public async Task<User> GetUserAsync(int id)
    {
        logger.LogInformation("Getting user {UserId}", id);
        return await userRepository.GetByIdAsync(id);
    }
}

// Simple Guard class for validation
public static class Guard
{
    public static class Against
    {
        public static void Null<T>(T value, string parameterName) where T : class
        {
            if (value == null)
                throw new ArgumentNullException(parameterName);
        }
    }
}

### 2. Service Lifetime Best Practices

In [ ]:
// ✅ GOOD: Proper service lifetime configuration
public static class ServiceLifetimeExamples
{
    public static void ConfigureServices(IServiceCollection services)
    {
        // SINGLETON: Stateless, expensive to create, thread-safe
        services.AddSingleton<IMemoryCache, MemoryCache>();
        services.AddSingleton<IHttpClientFactory, HttpClientFactory>();
        services.AddSingleton<ICacheService, RedisCacheService>();
        
        // SCOPED: Per-request lifetime, may hold state for request
        services.AddScoped<DbContext, ApplicationDbContext>();
        services.AddScoped<IUserRepository, SqlUserRepository>();
        services.AddScoped<IUserService, UserService>();
        services.AddScoped<IUnitOfWork, UnitOfWork>();
        
        // TRANSIENT: Lightweight, stateless, new instance each time
        services.AddTransient<IMapper, AutoMapperService>();
        services.AddTransient<IValidator<User>, UserValidator>();
        services.AddTransient<ICommandHandler<CreateUserCommand>, CreateUserCommandHandler>();
        
        // Configuration objects - typically singleton
        services.Configure<EmailConfiguration>(config => { /* configure */ });
        services.AddSingleton<EmailConfiguration>(provider =>
            provider.GetRequiredService<IOptions<EmailConfiguration>>().Value);
        
        // Factory services for creating other services
        services.AddSingleton<IPaymentProcessorFactory, PaymentProcessorFactory>();
    }
}

// Example of a service that should be Singleton
public class RedisCacheService : ICacheService
{
    private readonly IConnectionMultiplexer _redis;
    private readonly IDatabase _database;
    
    public RedisCacheService(IConnectionMultiplexer redis)
    {
        _redis = redis; // Expensive connection, should be shared
        _database = _redis.GetDatabase();
    }
    
    // Thread-safe operations
    public async Task<T> GetAsync<T>(string key) => 
        JsonSerializer.Deserialize<T>(await _database.StringGetAsync(key));
    
    public async Task SetAsync<T>(string key, T value, TimeSpan expiration) =>
        await _database.StringSetAsync(key, JsonSerializer.Serialize(value), expiration);
}

// Example of a service that should be Scoped
public class UnitOfWork : IUnitOfWork, IDisposable
{
    private readonly DbContext _context;
    private readonly Dictionary<Type, object> _repositories = new();
    
    public UnitOfWork(DbContext context)
    {
        _context = context;
    }
    
    public IRepository<T> GetRepository<T>() where T : class
    {
        if (_repositories.ContainsKey(typeof(T)))
        {
            return (IRepository<T>)_repositories[typeof(T)];
        }
        
        var repository = new Repository<T>(_context);
        _repositories.Add(typeof(T), repository);
        return repository;
    }
    
    public async Task<int> SaveChangesAsync() => await _context.SaveChangesAsync();
    
    public void Dispose() => _context?.Dispose();
}

// Example of a service that should be Transient
public class UserValidator : IValidator<User>
{
    public ValidationResult Validate(User user)
    {
        var result = new ValidationResult();
        
        if (string.IsNullOrWhiteSpace(user.Email))
            result.Errors.Add("Email is required");
        
        if (!IsValidEmail(user.Email))
            result.Errors.Add("Email format is invalid");
        
        result.IsValid = !result.Errors.Any();
        return result;
    }
    
    private bool IsValidEmail(string email) => 
        !string.IsNullOrWhiteSpace(email) && email.Contains('@');
}

// Supporting interfaces and classes
public interface ICacheService
{
    Task<T> GetAsync<T>(string key);
    Task SetAsync<T>(string key, T value, TimeSpan expiration);
}

public interface IUnitOfWork
{
    IRepository<T> GetRepository<T>() where T : class;
    Task<int> SaveChangesAsync();
}

public interface IRepository<T> where T : class { }
public class Repository<T> : IRepository<T> where T : class
{
    public Repository(DbContext context) { }
}

public interface IValidator<T>
{
    ValidationResult Validate(T item);
}

public class ValidationResult
{
    public bool IsValid { get; set; }
    public List<string> Errors { get; set; } = new();
}

public interface IMapper { }
public class AutoMapperService : IMapper { }
public interface ICommandHandler<T> { }
public class CreateUserCommand { }
public class CreateUserCommandHandler : ICommandHandler<CreateUserCommand> { }
public interface IPaymentProcessorFactory { }
public class PaymentProcessorFactory : IPaymentProcessorFactory { }

## 📝 **Key Takeaways**

### ✅ **Use Dependency Injection When:**
- Creating abstraction layers for better testability
- Managing complex dependencies between services
- Implementing different behaviors based on configuration
- Applying cross-cutting concerns (logging, caching, timing)
- Following SOLID principles

### ❌ **Don't Use DI When:**
- Injecting simple value types or constants
- Creating unnecessary abstractions for simple operations
- The class has too many dependencies (SRP violation)
- Using Service Locator pattern instead of proper injection

### 🔧 **Best Practices:**
1. **Prefer constructor injection** over property injection
2. **Consider validating all constructor parameters** for null values
3. **Choose appropriate service lifetimes** (Singleton, Scoped, Transient)
4. **Use interfaces** for better abstraction and testing
5. **Minimize the 'new' keyword** in business logic
6. **Register services in the composition root** (Startup/Program)
7. **Use factories** for complex object creation scenarios
8. **Implement decorators** for cross-cutting concerns

### 🎯 **Service Lifetime Guidelines:**
- **Singleton**: Stateless, expensive to create, thread-safe services
- **Scoped**: Per-request services, database contexts, unit of work
- **Transient**: Lightweight, stateless services, validators, mappers

### 🧪 **Testing Benefits:**
- Easy mocking of dependencies
- Isolated unit testing
- Integration testing with test doubles
- Configuration-based behavior switching